# Getting Started with DIP-SMC-PSO

**Welcome!** This notebook provides an interactive introduction to the Double Inverted Pendulum Sliding Mode Control with PSO Optimization framework.

## What You'll Learn

1. **Simulate the system** with different controllers
2. **Compare controller performance** visually
3. **Run PSO optimization** to tune gains
4. **Visualize results** with interactive plots
5. **Analyze stability** and control effectiveness

---

## Table of Contents

- [Setup](#setup)
- [Quick Simulation](#quick-simulation)
- [Controller Comparison](#controller-comparison)
- [PSO Optimization](#pso-optimization)
- [Advanced Analysis](#advanced-analysis)
- [Next Steps](#next-steps)

---

## Setup

First, let's import the necessary modules and configure the environment.

In [ ]:
# Standard library imports
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root))

# Third-party imports
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Project imports
from src.controllers.factory import create_controller
from src.core.dynamics import SimplifiedDynamics
from src.core.simulation_runner import SimulationRunner
from src.config import load_config

# Configure matplotlib for notebook
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Setup complete!")
print(f"✓ Project root: {project_root}")

---

## Quick Simulation

Let's run a quick 5-second simulation with the **Classical SMC** controller.

In [ ]:
# Load configuration
config = load_config("config.yaml")

# Create controller and dynamics
controller = create_controller('classical_smc', config=config)
dynamics = SimplifiedDynamics(config.physics)

# Run simulation
print("Running simulation...")
sim = SimulationRunner(controller, dynamics, config.simulation)
result = sim.run()

print(f"✓ Simulation complete!")
print(f"  Duration: {result.time[-1]:.2f} seconds")
print(f"  Final cart position: {result.states[-1, 0]:.4f} m")
print(f"  Final angle 1: {np.degrees(result.states[-1, 2]):.2f}°")
print(f"  Final angle 2: {np.degrees(result.states[-1, 4]):.2f}°")

### Visualize Results

Let's plot the system states over time.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Cart position
axes[0].plot(result.time, result.states[:, 0], 'b-', linewidth=2)
axes[0].set_ylabel('Cart Position (m)', fontsize=12)
axes[0].set_title('Classical SMC - System Response', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Pendulum angles
axes[1].plot(result.time, np.degrees(result.states[:, 2]), 'r-', label='θ₁ (lower)', linewidth=2)
axes[1].plot(result.time, np.degrees(result.states[:, 4]), 'g-', label='θ₂ (upper)', linewidth=2)
axes[1].set_ylabel('Angle (degrees)', fontsize=12)
axes[1].legend(loc='upper right', fontsize=10)
axes[1].grid(True, alpha=0.3)

# Control force
axes[2].plot(result.time, result.controls, 'm-', linewidth=2)
axes[2].set_ylabel('Control Force (N)', fontsize=12)
axes[2].set_xlabel('Time (s)', fontsize=12)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Plots generated successfully!")

---

## Controller Comparison

Let's compare **4 different controllers** on the same initial conditions:

1. **Classical SMC** - Standard sliding mode control
2. **STA-SMC** - Super-twisting algorithm (chattering reduction)
3. **Adaptive SMC** - Adapts to system uncertainties
4. **Hybrid Adaptive STA-SMC** - Best of both worlds

In [ ]:
# Controllers to compare
controller_names = ['classical_smc', 'sta_smc', 'adaptive_smc', 'hybrid_adaptive_sta_smc']
colors = ['blue', 'red', 'green', 'purple']
results = {}

# Run simulations
print("Running controller comparison...")
for name in controller_names:
    print(f"  - {name}...", end=" ")
    controller = create_controller(name, config=config)
    sim = SimulationRunner(controller, dynamics, config.simulation)
    results[name] = sim.run()
    print("✓")

print("\n✓ All simulations complete!")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Controller Comparison', fontsize=16, fontweight='bold')

# Cart position
for (name, result), color in zip(results.items(), colors):
    axes[0, 0].plot(result.time, result.states[:, 0], color=color, label=name.replace('_', ' ').title(), linewidth=2)
axes[0, 0].set_ylabel('Cart Position (m)', fontsize=11)
axes[0, 0].set_title('Cart Position', fontsize=12, fontweight='bold')
axes[0, 0].legend(loc='best', fontsize=9)
axes[0, 0].grid(True, alpha=0.3)

# Angle 1
for (name, result), color in zip(results.items(), colors):
    axes[0, 1].plot(result.time, np.degrees(result.states[:, 2]), color=color, label=name.replace('_', ' ').title(), linewidth=2)
axes[0, 1].set_ylabel('Angle θ₁ (degrees)', fontsize=11)
axes[0, 1].set_title('Lower Pendulum Angle', fontsize=12, fontweight='bold')
axes[0, 1].legend(loc='best', fontsize=9)
axes[0, 1].grid(True, alpha=0.3)

# Angle 2
for (name, result), color in zip(results.items(), colors):
    axes[1, 0].plot(result.time, np.degrees(result.states[:, 4]), color=color, label=name.replace('_', ' ').title(), linewidth=2)
axes[1, 0].set_ylabel('Angle θ₂ (degrees)', fontsize=11)
axes[1, 0].set_xlabel('Time (s)', fontsize=11)
axes[1, 0].set_title('Upper Pendulum Angle', fontsize=12, fontweight='bold')
axes[1, 0].legend(loc='best', fontsize=9)
axes[1, 0].grid(True, alpha=0.3)

# Control effort
for (name, result), color in zip(results.items(), colors):
    axes[1, 1].plot(result.time, result.controls, color=color, label=name.replace('_', ' ').title(), linewidth=2)
axes[1, 1].set_ylabel('Control Force (N)', fontsize=11)
axes[1, 1].set_xlabel('Time (s)', fontsize=11)
axes[1, 1].set_title('Control Force', fontsize=12, fontweight='bold')
axes[1, 1].legend(loc='best', fontsize=9)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Performance Metrics

Let's compute quantitative performance metrics for each controller.

In [ ]:
import pandas as pd

# Compute metrics
metrics_data = []
for name, result in results.items():
    # Settling time (when angle stays within 1 degree)
    angle_error = np.abs(np.degrees(result.states[:, 2]))
    settled_idx = np.where(angle_error < 1.0)[0]
    settling_time = result.time[settled_idx[0]] if len(settled_idx) > 0 else np.nan
    
    # Control effort (integral of squared control)
    control_effort = np.trapz(result.controls**2, result.time)
    
    # Maximum angle deviation
    max_angle = np.max(np.abs(np.degrees(result.states[:, 2])))
    
    # Final position error
    final_pos_error = np.abs(result.states[-1, 0])
    
    metrics_data.append({
        'Controller': name.replace('_', ' ').title(),
        'Settling Time (s)': f"{settling_time:.3f}" if not np.isnan(settling_time) else "N/A",
        'Control Effort': f"{control_effort:.1f}",
        'Max Angle (deg)': f"{max_angle:.2f}",
        'Final Pos Error (m)': f"{final_pos_error:.4f}"
    })

# Display as table
df = pd.DataFrame(metrics_data)
display(HTML(df.to_html(index=False)))

print("\n✓ Performance metrics computed!")

---

## PSO Optimization

Now let's use **Particle Swarm Optimization (PSO)** to automatically tune controller gains for optimal performance.

⚠️ **Note:** PSO optimization can take several minutes. We'll use reduced parameters for demonstration.

In [ ]:
from src.optimizer.pso_optimizer import PSOTuner

# Create PSO tuner
print("Setting up PSO optimization...")
pso_config = config.pso
pso_config.max_iterations = 20  # Reduced for demo (default: 100)
pso_config.num_particles = 15   # Reduced for demo (default: 30)

# Define parameter bounds for Classical SMC (6 gains)
bounds = (
    np.array([5.0, 3.0, 5.0, 2.0, 10.0, 1.0]),   # Lower bounds
    np.array([20.0, 10.0, 15.0, 8.0, 30.0, 8.0])  # Upper bounds
)

# Create tuner
tuner = PSOTuner(
    controller_type='classical_smc',
    config=config,
    bounds=bounds
)

print("✓ PSO tuner ready")
print(f"  Particles: {pso_config.num_particles}")
print(f"  Iterations: {pso_config.max_iterations}")
print(f"  Parameter space: 6D (6 controller gains)")

In [ ]:
# Run optimization
print("\nRunning PSO optimization...")
print("This may take 2-3 minutes...\n")

best_gains, best_fitness = tuner.optimize()

print("\n" + "="*60)
print("✓ Optimization Complete!")
print("="*60)
print(f"\nBest Fitness: {best_fitness:.6f}")
print(f"\nOptimized Gains:")
for i, gain in enumerate(best_gains, 1):
    print(f"  Gain {i}: {gain:.4f}")

### Compare Original vs Optimized

Let's see how the optimized controller performs compared to the default gains.

In [ ]:
# Run with optimized gains
controller_optimized = create_controller('classical_smc', config=config, gains=best_gains)
sim_optimized = SimulationRunner(controller_optimized, dynamics, config.simulation)
result_optimized = sim_optimized.run()

# Run with default gains
controller_default = create_controller('classical_smc', config=config)
sim_default = SimulationRunner(controller_default, dynamics, config.simulation)
result_default = sim_default.run()

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(12, 8))
fig.suptitle('PSO Optimization Results', fontsize=16, fontweight='bold')

# Angle comparison
axes[0].plot(result_default.time, np.degrees(result_default.states[:, 2]), 'b--', label='Default Gains', linewidth=2, alpha=0.7)
axes[0].plot(result_optimized.time, np.degrees(result_optimized.states[:, 2]), 'r-', label='Optimized Gains (PSO)', linewidth=2)
axes[0].set_ylabel('Angle θ₁ (degrees)', fontsize=12)
axes[0].set_title('Lower Pendulum Angle', fontsize=13, fontweight='bold')
axes[0].legend(loc='upper right', fontsize=11)
axes[0].grid(True, alpha=0.3)

# Control effort comparison
axes[1].plot(result_default.time, result_default.controls, 'b--', label='Default Gains', linewidth=2, alpha=0.7)
axes[1].plot(result_optimized.time, result_optimized.controls, 'r-', label='Optimized Gains (PSO)', linewidth=2)
axes[1].set_ylabel('Control Force (N)', fontsize=12)
axes[1].set_xlabel('Time (s)', fontsize=12)
axes[1].set_title('Control Force', fontsize=13, fontweight='bold')
axes[1].legend(loc='upper right', fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Comparison plots generated!")

---

## Advanced Analysis

Let's perform some advanced analysis on controller performance.

### Phase Portrait

A phase portrait shows the system's trajectory in state space (angle vs angular velocity).

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Phase Portraits', fontsize=16, fontweight='bold')

# Lower pendulum phase portrait
axes[0].plot(np.degrees(result_optimized.states[:, 2]), np.degrees(result_optimized.states[:, 3]), 'r-', linewidth=2)
axes[0].plot(np.degrees(result_optimized.states[0, 2]), np.degrees(result_optimized.states[0, 3]), 'go', markersize=10, label='Start')
axes[0].plot(np.degrees(result_optimized.states[-1, 2]), np.degrees(result_optimized.states[-1, 3]), 'rs', markersize=10, label='End')
axes[0].set_xlabel('Angle θ₁ (degrees)', fontsize=12)
axes[0].set_ylabel('Angular Velocity dθ₁/dt (deg/s)', fontsize=12)
axes[0].set_title('Lower Pendulum', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Upper pendulum phase portrait
axes[1].plot(np.degrees(result_optimized.states[:, 4]), np.degrees(result_optimized.states[:, 5]), 'b-', linewidth=2)
axes[1].plot(np.degrees(result_optimized.states[0, 4]), np.degrees(result_optimized.states[0, 5]), 'go', markersize=10, label='Start')
axes[1].plot(np.degrees(result_optimized.states[-1, 4]), np.degrees(result_optimized.states[-1, 5]), 'rs', markersize=10, label='End')
axes[1].set_xlabel('Angle θ₂ (degrees)', fontsize=12)
axes[1].set_ylabel('Angular Velocity dθ₂/dt (deg/s)', fontsize=12)
axes[1].set_title('Upper Pendulum', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Phase portraits show convergence to equilibrium (0, 0)")

### Energy Analysis

Let's analyze the system's energy dissipation over time.

In [ ]:
# Physical parameters
m_cart = config.physics.m_cart
m1 = config.physics.m1
m2 = config.physics.m2
l1 = config.physics.l1
l2 = config.physics.l2
g = 9.81

# Compute kinetic and potential energy
states = result_optimized.states
x_dot = states[:, 1]
theta1_dot = states[:, 3]
theta2_dot = states[:, 5]
theta1 = states[:, 2]
theta2 = states[:, 4]

# Kinetic energy (simplified)
KE = 0.5 * (m_cart * x_dot**2 + m1 * (l1 * theta1_dot)**2 + m2 * (l2 * theta2_dot)**2)

# Potential energy (simplified)
PE = m1 * g * l1 * (1 - np.cos(theta1)) + m2 * g * l2 * (1 - np.cos(theta2))

# Total energy
E_total = KE + PE

# Plot
fig, axes = plt.subplots(2, 1, figsize=(12, 8))
fig.suptitle('Energy Analysis', fontsize=16, fontweight='bold')

# Energy components
axes[0].plot(result_optimized.time, KE, 'b-', label='Kinetic Energy', linewidth=2)
axes[0].plot(result_optimized.time, PE, 'r-', label='Potential Energy', linewidth=2)
axes[0].plot(result_optimized.time, E_total, 'k--', label='Total Energy', linewidth=2)
axes[0].set_ylabel('Energy (J)', fontsize=12)
axes[0].set_title('Energy Components', fontsize=13, fontweight='bold')
axes[0].legend(loc='upper right', fontsize=11)
axes[0].grid(True, alpha=0.3)

# Energy dissipation rate
energy_rate = np.gradient(E_total, result_optimized.time)
axes[1].plot(result_optimized.time, energy_rate, 'm-', linewidth=2)
axes[1].set_ylabel('Energy Rate (J/s)', fontsize=12)
axes[1].set_xlabel('Time (s)', fontsize=12)
axes[1].set_title('Energy Dissipation Rate', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✓ Initial energy: {E_total[0]:.4f} J")
print(f"✓ Final energy: {E_total[-1]:.6f} J")
print(f"✓ Energy dissipated: {E_total[0] - E_total[-1]:.4f} J")

---

## Next Steps

Congratulations! You've completed the getting-started tutorial. Here's what to explore next:

### 1. Advanced Controllers
- Try the **MPC controller** for predictive control
- Experiment with **swing-up control** from hanging position
- Implement your own custom controller

### 2. Optimization
- Run full PSO optimization (100 iterations, 30 particles)
- Explore multi-objective optimization
- Compare different optimization algorithms

### 3. Analysis
- Monte Carlo robustness analysis
- Lyapunov stability validation
- Performance benchmarking

### 4. Hardware Integration
- Try **Hardware-in-the-Loop (HIL)** simulation
- Real-time control experiments
- Deploy to embedded systems

### 5. Documentation
- **Theory Guide**: `docs/theory/` - Mathematical foundations
- **API Reference**: `docs/api/` - Detailed API docs
- **How-To Guides**: `docs/how_to/` - Step-by-step tutorials
- **Research Workflow**: `docs/workflow/` - Research best practices

---

### Resources

- 📖 [Full Documentation](https://dip-smc-pso.readthedocs.io/)
- 🐙 [GitHub Repository](https://github.com/theSadeQ/dip-smc-pso)
- 💬 [Discussions](https://github.com/theSadeQ/dip-smc-pso/discussions)
- 🐛 [Issue Tracker](https://github.com/theSadeQ/dip-smc-pso/issues)

---

**Happy experimenting!** 🚀